<a href="https://colab.research.google.com/github/GeorgeNishanth/NG/blob/main/Copy_of_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [ ]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()

# Read the file (make sure the filename matches)
data = pd.read_csv('IMDB Dataset.csv')

# Display the data
print(data.head())

Saving IMDB Dataset.csv to IMDB Dataset (1).csv
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [ ]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [ ]:
import nltk
nltk.download('stopwords')  # This will download the stopwords dataset

from nltk.corpus import stopwords
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


<ipython-input-12-5f078287338f>:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
26872    [six, stars, paul, newman, portrayal, general,...
7178     [reason, gave, movie, couple, reasons, movie, ...
1240     [could, considered, mild, spoilers, anyone, el...
15704    [sandra, bullock, paints, believable, picture,...
32980    [spoilers, spoilers, the, cell, four, do, beli...
                               ...                        
17900    [one, excellent, movies, ever, produced, russi...
37258    [hadnt, heard, lot, movie, except, national, a...
29360    [mature, intelligent, highly, charged, melodra...
21381    [i, must, admit, type, film, i, would, normall...
40789    [most, stoogephiles, consider, best, stooges, ...
Name: review, Length: 40000, dtype: object 

18490    [i, surprised, much, i, enjoyed, sure, bit, sl...
16370    [this, movie, wastes, virtually, every, actor,...
22152    [a, poorly, written, script, likeable, charact...
27689    [city, hall, takes, politics, city, rather, co...
20947    [this, movie, horrible, see, i, discuss, entir...
 

In [ ]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [ ]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[ 1310   314   723 ...     0     0     0]
 [  196   421     3 ...     0     0     0]
 [   27  1091  3647 ...     1    58    55]
 ...
 [ 2527  1000   450 ...     0     0     0]
 [    1   116   890 ...   880  5729     7]
 [  725 92414  1066 ...     0     0     0]] 

Encoded X Test
 [[    1   652    17 ...     0     0     0]
 [    8     3  6699 ...     0     0     0]
 [   39   781   311 ...     0     0     0]
 ...
 [    1   195 41396 ...     0     0     0]
 [    2  2873   569 ...   382   934   133]
 [    1    12    58 ...     0     0     0]] 

Maximum review length:  130


In [ ]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
checkpoint = ModelCheckpoint(
    'models/LSTM.keras',  # Change .h5 to .keras
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [ ]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - accuracy: 0.5553 - loss: 0.6767
Epoch 1: accuracy improved from -inf to 0.60375, saving model to models/LSTM.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 76s 230ms/step - accuracy: 0.5555 - loss: 0.6766
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.6194 - loss: 0.6502
Epoch 2: accuracy improved from 0.60375 to 0.60883, saving model to models/LSTM.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 233ms/step - accuracy: 0.6194 - loss: 0.6502
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - accuracy: 0.6071 - loss: 0.6367
Epoch 3: accuracy improved from 0.60883 to 0.62007, saving model to models/LSTM.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 70s 225ms/step - accuracy: 0.6072 - loss: 0.6367
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - accuracy: 0.6393 - loss: 0.6203
Epoch 4: accuracy improved from 0.62007 to 0.63877, saving model to models/LSTM.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 85s 233ms/step - accuracy: 0.6393 - loss: 0.6204

In [ ]:
y_pred = (model.predict(x_test, batch_size=128) > 0.5).astype('int32')  # Convert probabilities to binary labels

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {:.2f}%'.format(true / len(y_pred) * 100))

79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 100ms/step
Correct Prediction: 6517
Wrong Prediction: 3483
Accuracy: 65.17%


In [ ]:
loaded_model = load_model('models/LSTM.keras')

In [ ]:

review = str(input('Movie Review: '))

Movie Review: This movie was anything but ordinary. Everything was expertly crafted — from the story and pacing to the setting and overall execution. I highly recommend it to mystery enthusiasts and anyone looking for a great film to watch!


In [ ]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  This movie was anything but ordinary Everything was expertly crafted  from the story and pacing to the setting and overall execution I highly recommend it to mystery enthusiasts and anyone looking for a great film to watch
Filtered:  ['this movie anything ordinary everything expertly crafted  story pacing setting overall execution i highly recommend mystery enthusiasts anyone looking great film watch']


In [ ]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[    8     3   141  1777   177  7371  2773    13  1713   866   345  2604
      1   450   281   680 10303   154   163    21     4    32     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0]]


In [ ]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 719ms/step
[[0.33582106]]


In [ ]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

negative
